In [10]:
from gurobipy import Model, GRB, quicksum
import numpy as np
from random import randint
from math import log10, floor

In [4]:
N =10
A = 5
alpha = 0.5
states = [s for s in range(1, N+1)]
actions = [a for a in range(1, A+1)]

n_states = len(states) + states[0]
n_actions = len(actions) + actions[0]
p = np.zeros((n_states, n_actions, n_states))
r = np.zeros((n_states, n_actions, n_states))

In [5]:
# initialize rewards
for s in states:
    for a in actions:
        reward = randint(0, 10)
        for s1 in states:
            r[s, a, s1] = reward

# initialize transition probability
for s in states:
    for a in actions:
        lst = [np.random.uniform() for s in range(len(states))]
        norm = [float(i) / sum(lst) for i in lst]
        for s1 in states:
            p[s, a, s1] = norm[(s1-1)]

### part.A)

In [34]:
def round_sig(x, sig=6):
    return round(x, sig-int(floor(log10(abs(x))))-1)

def policy_iter(states, actions, prob, rewrd, alpha):
    n_states = len(states) + states[0]
    n_actions = len(actions) + actions[0]
    # initialize policy
    policy = [1 for s in range(n_states)]
    v = dict()
    # initialize v for each state
    for s in states:
        v[s] = 0

    updated = True
    iterations = 0
    while updated:
        updated = False
        iterations += 1
        # update value function by value iteration
        for s in states:
            v[s] = sum([prob[s, policy[s], s1]*(rewrd[s, policy[s], s1] + alpha*v[s1]) for s1 in states])

        for s in states:
            q_best = v[s]
            for a in actions:
                q_sa = sum([prob[s, a, s1] * (rewrd[s, a, s1] + alpha*v[s1]) for s1 in states])
                if q_sa > q_best:
                    policy[s] = a
                    q_best = q_sa
                    updated = True
    
    # keep 4 significant numbers
    for s in states:
        v[s] = round_sig(v[s])
    print("policy improvment method: the optimal value functions are:{}, the optimal policy are: {}".format(v,policy[states[0]:]))

### part.B)

In [18]:
mdl = Model('PI')

Using license file C:\Users\Vance He\gurobi.lic
Academic license - for non-commercial use only


In [19]:
x = mdl.addVars(states, vtype=GRB.CONTINUOUS)
mdl.update()

In [20]:
mdl.setObjective(quicksum(x[i] for i in states), GRB.MINIMIZE)

In [21]:
for i in states:
    for a in actions:
        mdl.addConstr(x[i] >= quicksum(p[i, a, j]*(r[i,a,j]+alpha*x[j])for j in states))

### part.C)

In [36]:
# Part B) result Gurobi
mdl.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 50 rows, 10 columns and 500 nonzeros
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]

Solved in 10 iterations and 0.00 seconds
Optimal objective  1.782267476e+02


In [37]:
# load results into list
lst = [0]
for v in mdl.getVars():
    lst.append(v.x)

In [42]:
# print value function
value_func = dict()

# keep 4 significant numbers
for s in states:
    lst[s] = round_sig(lst[s])
for s in states:
    value_func[s] = lst[s] 

In [43]:
policy = []
for i in states:
    for a in actions:
        if (lst[i] -sum([p[i, a, j]*(r[i,a,j]+alpha*lst[j]) for j in states])) < 0.01:
            policy.append(a)

In [39]:
# optimal value function
mdl.printAttr('x')


    Variable            x 
-------------------------
          C0       18.967 
          C1      17.8753 
          C2      18.8445 
          C3      18.9514 
          C4      19.0108 
          C5      14.0049 
          C6      16.8032 
          C7      18.9316 
          C8      17.9234 
          C9      16.9147 


In [44]:
# part A) result
policy_iter(states, actions, p, r, alpha)

policy improvment method: the optimal value functions are:{1: 18.967, 2: 17.8753, 3: 18.8445, 4: 18.9514, 5: 19.0108, 6: 14.0049, 7: 16.8032, 8: 18.9316, 9: 17.9234, 10: 16.9147}, the optimal policy are: [1, 3, 1, 2, 2, 5, 3, 4, 1, 3]


In [45]:
# Part B) result
print("linear programming: the optimal value functions are:{}, the optimal policy are: {}".format(value_func ,policy))

linear programming: the optimal value functions are:{1: 18.967, 2: 17.8753, 3: 18.8445, 4: 18.9514, 5: 19.0108, 6: 14.0049, 7: 16.8032, 8: 18.9316, 9: 17.9234, 10: 16.9147}, the optimal policy are: [1, 3, 1, 2, 2, 5, 3, 4, 1, 3]


***From the two results above, we could see that these two methods produce the same result.*** 